# Introduction

**Introduction to the Project**

Welcome to our Car Accident Analysis project!

Our team, consisting of Yicheng Shen, Leshan Feng, and Yuanpeng Jia, has set out to analyze a comprehensive dataset of car accidents across the United States in order to identify key factors influencing accident occurrence and severity. By leveraging this dataset, which covers 49 states and contains around 2.8 million accident records, we aim to develop a model that can accurately predict the likelihood and severity of car accidents.

**Data scource**

The dataset, sourced from https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents, is a rich resource that includes variables such as accident severity, time, location, and descriptions. It has been compiled using multiple APIs that provide streaming traffic incident data from various sources, including the US and state departments of transportation, law enforcement agencies, traffic cameras, and road-network traffic sensors.

**Project design**

In this project, we will explore various statistical and machine learning models, including different supervised and unsupervised approaches to analyse the factors that effect the severity of the car accident. We will first use unsupervised machine learning method to reduce the number of features. Then we will compare the result of different supervised machine learning models. We aim to present the machine learning model that predict the severity of the accident, which can be used to promote safer driving habits.

The framework of the project is below:

Part 1: Data Cleaning and Preprocessing

Part 2: Exploratory data analysis (EDA)

Part 3: Machine Learning Modeling (Unsupervised and supervised)

Part 4: Model Comparison and Conclusion

Part 5: Challenges and Obstacles Faced

Part 6: Further Exploration

# Part 0 : Before Running⚠️

Remember to upload your kaggle.json file (kaggle account required) to colab after you run the following channel. Or, you can directly download file from https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents and upload to colab.

Alternately, if you don't want to run it, make sure to load it in Colab to have a complete view of our data visualization.



## Installing the Kaggle API in Colab

The dataset we use from kaggle. If you wish to run this notebook, you can do following steps:


First, grab your token from Kaggle.
1. Navigate to https://www.kaggle.com.
2. Then go to the [Account tab of your user profile](https://www.kaggle.com/me/account).
3. select Create API Token. This will trigger the download of kaggle.json, a file containing your API credentials.

Then run the cell below to upload kaggle.json to your Colab runtime.

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create the kaggle directory
# (NOTE: Do NOT run this cell more than once unless restarting kernel)
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# Read the uploaded kaggle.json file
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
# Download dataset
!!kaggle datasets download -d sobhanmoosavi/us-accidents

['us-accidents.zip: Skipping, found more recently modified local copy (use --force to force download)']

In [ ]:
# Unzip folder in Colab content folder
!unzip /content/us-accidents.zip

Archive:  /content/us-accidents.zip
replace US_Accidents_Dec21_updated.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import packages
import json
import glob
import pandas as pd
import numpy as np
import datetime as dt
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from google.colab import drive
from sklearn.model_selection import train_test_split
from collections import Counter
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.model_selection import train_test_split
import torch
import torchvision
from torchvision import transforms, utils
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from collections import Counter
from PIL import Image
from skimage import io, transform
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from google.colab import drive
torch.manual_seed(42) # For grading consistency
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(device)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

## Data Loading

We are loading the US_accident csv file that we downloaded from Kaggle.

Load `US_Accidents_Dec21_updated.csv` as `df_accident`.

In [ ]:
#read the dataset in
df_accident = pd.read_csv('US_Accidents_Dec21_updated.csv')

In [ ]:
#sneak peak of the dataset
df_accident.head(5)

In [ ]:
#number of rows and columns in the dataset
print('The Dataset Contains, Rows: {:,d} & Columns: {}'.format(df_accident.shape[0], df_accident.shape[1]))

In [ ]:
df_dtypes = df_accident.dtypes.reset_index()
df_dtypes.columns = ["Column Name", "Column Type"]
df_dtypes#check each column's variable type

# Part 1: Data Cleaning and Preprocessing

We are going to clean and preprocess our `df_accident` in this part.


In [ ]:
# convert the Start_Time & End_Time Variable into Datetime objects and create two new columns
df_accident['start_time'] = pd.to_datetime(df_accident.Start_Time,errors='coerce')
df_accident['end_time'] = pd.to_datetime(df_accident.End_Time,errors='coerce')

In [ ]:
# check the number of rows with null values in each column
null_counts = df_accident.isnull().sum()
print(null_counts)

In order to avoid too much observations and create extremely long run times for the system, we decided to select 50000 rows randomly.

In [ ]:
#drop N/A values and any Unamed columns in the dataframe
clean_df = df_accident.dropna()
#randomly select 50000 observations from the dataset to avoid extremely long run time for future models
clean_df = clean_df.sample(n=50000, random_state=42)
#clean_df = clean_df.drop('Unnamed: 0', axis=1)

In [ ]:
#check the number of states in the dataset
states = clean_df.State.unique()

In [ ]:
# Extract year, month, day, hour and weekday of both start time and end time and create new columns in the dataset
clean_df['Year'] = clean_df['start_time'].dt.year
clean_df['Month'] = clean_df['start_time'].dt.strftime('%b')
clean_df['Day'] = clean_df['start_time'].dt.day
clean_df['Hour'] = clean_df['start_time'].dt.hour
clean_df['Weekday'] = clean_df['start_time'].dt.strftime('%a')
clean_df['Year_end'] = clean_df['end_time'].dt.year
clean_df['Month_end'] = clean_df['end_time'].dt.strftime('%b')
clean_df['Day_end'] = clean_df['end_time'].dt.day
clean_df['Hour_end'] = clean_df['end_time'].dt.hour
clean_df['Weekday_end'] = clean_df['end_time'].dt.strftime('%a')


In [ ]:
# Extract the amount of time in the unit of minutes for each accident, round to the nearest integer
td='Time_Duration(min)'
clean_df[td]=round((clean_df['end_time']-clean_df['start_time'])/np.timedelta64(1,'m'))

In [ ]:
#define the negative accident time as outliers
outliers=clean_df[td]<=0

# Set outlying accident time to NAN
clean_df[outliers] = np.nan

# Drop rows with NaN values
clean_df.dropna(subset=[td],axis=0,inplace=True)

Create a new column called `severe_accident`

In [ ]:
#divide accident's severity into two categories, if the severity is 1 or 2,
#it will be categorized as not severe, else it is severe
severity = lambda x: 'no' if x in [1, 2] else 'yes'

# apply the lambda function to create a new column
clean_df['severe_accident'] = clean_df['Severity'].apply(severity)

# Part 2: Exploratory data analysis (EDA): Data Visualization

## Handling Missing Data

In [ ]:
missing_data = df_accident.isnull().sum()
missing_data_percentage = (missing_data / len(df_accident)) * 100
print("Missing Data Percentage: \n", missing_data_percentage)


## Get Accident counts


In [ ]:
df_state_counts = pd.DataFrame(clean_df.groupby(['State'])['ID'].count())
df_state_counts = df_state_counts.reset_index()
df_state_counts = df_state_counts.rename(columns={'ID': 'total counts'})
df_state_counts = df_state_counts.sort_values('total counts', ascending=False)

In [ ]:
df_state_counts #gives a clear view of which state has the most accidents and least accidents

## Identify which states have the most number of accidents with visualization


In [ ]:
counts_by_states=[]
for i in clean_df.State.unique():
    counts_by_states.append(clean_df[clean_df['State']==i].count()['ID'])

fig,ax = plt.subplots(figsize=(10,8))
ax = sns.barplot(x=states, y=counts_by_states, ax=ax)
ax.set(xlabel='State Names', ylabel='Number of Accidents', title = 'Numbers of Accidents in Different States')
plt.xticks(rotation = 45)
plt.show()

## check for the top 5 weather conditions that can cause accidents to happen with visualization


In [ ]:
fig, ax=plt.subplots(figsize=(6,6))
clean_df['Weather_Condition'].value_counts().sort_values(ascending=False).head(5).plot.bar(width=0.5,edgecolor='k',align='center',linewidth=2)
plt.xlabel('Weather_Condition')
plt.ylabel('Number of Accidents')
ax.tick_params(labelsize=20)
plt.title('5 Top Weather Condition for accidents')
plt.show()

## Map visualization

In [ ]:
import plotly.express as px

# We will limit the data to a smaller sample for faster visualization
sample_df = df_accident.sample(5000)

fig = px.scatter_geo(sample_df,
                     lat='Start_Lat',
                     lon='Start_Lng',
                     color='Severity',
                     hover_name='Description',
                     size='Severity',
                     scope='usa',
                     title='Accident Locations and Severity in the United States')
fig.show()


**Severity**: examine the relationship between accident severity and time duration

In [ ]:
# Analyze accident severity
sns.countplot(x='Severity', data=df_accident)
plt.title("Accident Severity Distribution")
plt.show()


In [ ]:
sns.boxplot(x='Severity', y='Time_Duration(min)', data=clean_df)
plt.title('Accident Severity by Time Duration')
plt.xlabel('Severity')
plt.ylabel('Time Duration (min)')
plt.show()


## Distribution of accident severity by state

In [ ]:
clean_df['State'] = clean_df['State'].astype(str)
state_severity = clean_df.groupby(['State', 'Severity']).size().reset_index(name='Counts')
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(16, 8))
sns.set(font_scale=1.2)

sns.scatterplot(x='State', y='Counts', hue='Severity', data=state_severity, s=100)
plt.title('Distribution of Accident Severity by State', fontsize=24)
plt.xlabel('State', fontsize=18)
plt.ylabel('Counts', fontsize=18)
plt.legend(title='Severity', fontsize=16, title_fontsize=16)
plt.show()


## Correlation between each variables(both independent and dependent)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize=(14, 10))
corr = clean_df.corr().round(2)
sns.heatmap(corr, cmap='coolwarm', annot=True, fmt='.2f', annot_kws={'fontsize': 12, 'fontweight': 'bold', 'color': 'black'}, ax=ax)
ax.set_title('Correlation between Variables', fontsize=16, fontweight='bold')
plt.show()



# Part 3: Machine Learning Modeling


## Unsupervised Learning Models

### **Hyperparameter Tunning: Using PCA**

In [ ]:
#Import necessary libraries
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Select the features to be used in the model
features_temp = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)',
            'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity',
            'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']

# Encode the categorical features using one-hot encoding
# X = pd.get_dummies(clean_df[features_temp], columns=['Weather_Condition'])
X = clean_df[features_temp]
# Extract the target variable
y = clean_df['Severity']

# Split the data into training and testing sets(80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X.shape

In [ ]:
#Intermediate step to address fact that PCA is not scale-invariant
scaler = StandardScaler()
scaler.fit(X_train)
# Transform
scaled_x_train = scaler.transform(X_train)
scaled_x_test = scaler.transform(X_test)

#Instantiate and Fit PCA
pca = PCA(n_components = 18)
pca_x_train = pca.fit(scaled_x_train)

In [ ]:
#Save the explained variance ratios into variable called "explained_variance_ratios"
explained_variance_ratios = pca.explained_variance_ratio_

#Save the CUMULATIVE explained variance ratios into variable called "cum_evr"
cum_evr = np.cumsum(explained_variance_ratios)

In [ ]:
# find optimal num components to use (n) by plotting explained variance ratio
plt.plot(np.arange(0, 18), cum_evr)
plt.plot(np.arange(0, 18), [0.95]*18)
plt.xlabel("# of components")
plt.ylabel("Proportion of Explained Variance")
plt.title("PVE as # of components increases")
plt.show()

In [ ]:
# Determine the number of components that explain 95% of the variance
optimal_components = np.argmax(cum_evr >= 0.95) + 1

# Print the optimal number of components
print(f"The optimal number of components is {optimal_components}")

**Based on the above plot, we can see the optimal number of components is around 66, to get a precise number. The np.argmax() function returns the index of the first occurrence of the maximum value in a given array. In this case, the cumulative variance ratio array contains the cumulative sum of explained variance ratios for each component. In this way, since python's index begins with 0 so we added 1 to get the number that represent the number of components as it starts with 1.**

In [ ]:
#Refit and transform on training with optimal n (as deduced from the previous step)
pca = PCA(n_components=optimal_components)
X_train_pca = pca.fit_transform(scaled_x_train)

#Transform on Testing Set and store it as `X_test_pca`
X_test_pca = pca.transform(scaled_x_test)

### **Logistic Regression with PCA**

In [ ]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics
# Initialize `log_reg_pca` model with default parameters and fit it on the PCA transformed training set
log_reg_pca = LogisticRegression(penalty = None, random_state=42, solver = 'saga',max_iter=50000)

# Use the model to predict on the PCA transformed test set and save these predictions as `y_pred`
log_reg_pca.fit(X_train_pca, y_train)
y_pred = log_reg_pca.predict(X_test_pca)
# Find the accuracy
test_accuracy = sklearn.metrics.accuracy_score(y_pred, y_test)

In [ ]:
print(f"The accuracy of this model is {test_accuracy}")

The accuracy of this model is 0.9452


### **Logistic Regression (whether an accident is severe or not) with PCA**

###**Logistic Regression with tuning: Lasso(L1), Ridge(L2), and ElasticNet(L1&L2)**

**LASSO L1**

In [ ]:
#LASSO L1
regr1 = LogisticRegression(penalty='l1', solver='saga', max_iter=50000, random_state=42)
#fit the model
regr1.fit(X_train_pca, y_train)

y_pred = regr1.predict(X_test_pca)
#compute test accuracy
test_accuracy = sklearn.metrics.accuracy_score(y_pred, y_test)
print(f"The test accuracy of this model is {test_accuracy}")

The test accuracy of this model is 0.9452


**Ridge L2**

In [ ]:
#Ridge L2
regr2 = LogisticRegression(penalty='l2', solver='saga', max_iter=50000, random_state=42)
#fit the model
regr2.fit(X_train_pca, y_train)

y_pred = regr2.predict(X_test_pca)
#compute test accuracy
test_accuracy = sklearn.metrics.accuracy_score(y_pred, y_test)
print(f"The test accuracy of this model is {test_accuracy}")

The test accuracy of this model is 0.9452


**L1 combined with L2 (Elastic Net)**

In [ ]:
#ElasticNet
regr3 = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=50000, l1_ratio=0.5, random_state=42)
#fit the model
regr3.fit(X_train_pca, y_train)

y_pred = regr3.predict(X_test_pca)
#compute test accuracy
test_accuracy = sklearn.metrics.accuracy_score(y_pred, y_test)
print(f"The test accuracy of this model is {test_accuracy}")

The test accuracy of this model is 0.9452


**Conclusion**:

By implementing different regularizations, we have found the model has accuracy of 94.52% by using Lasso, Ridge, and elastic net, and the result are basically the same compared to pure logistic regression. This indicates that, in this specific problem, adding regularization does not provide significant improvements in model performance.


## Supervised Learning Models (predicting the severity of an accident)
Predicting the severity of accidents using machine learning models can help improve road safety and save lives. Accidents can have different levels of severity, ranging from minor injuries to fatalities, and understanding the factors that contribute to these outcomes can inform prevention strategies and improve emergency response. By analyzing large datasets of accident reports and identifying patterns and risk factors associated with different levels of severity, machine learning models can be trained to accurately predict the severity of future accidents. This information can be used to inform policies and interventions aimed at reducing the risk of accidents and mitigating their impact. Overall, the development of machine learning models for accident severity prediction can have important implications for public safety and welfare.

###**Decision tree**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Define features and target
X = clean_df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)',
            'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity', 'Bump',
            'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
            'Roundabout', 'Station', 'Stop',
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]
y = clean_df['Severity']

# Split the data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the decision tree model
model = DecisionTreeClassifier(random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy score and Cohen's kappa for the baseline model
acc_score = accuracy_score(y_test, y_pred)
kappa_score = cohen_kappa_score(y_test, y_pred)
print("Baseline Model:")
print("Accuracy Score:", acc_score)
print("Cohen's Kappa Score:", kappa_score)

# Hold one out (90/10 train/test split)
X_train_90, X_test_90, y_train_90, y_test_90 = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train_90, y_train_90)
y_pred_90 = model.predict(X_test_90)
acc_score_90 = accuracy_score(y_test_90, y_pred_90)
kappa_score_90 = cohen_kappa_score(y_test_90, y_pred_90)
print("\nHold one out (90/10 train/test split):")
print("Accuracy Score:", acc_score_90)
print("Cohen's Kappa Score:", kappa_score_90)

# 5-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=5)
acc_score_cv = cv_scores.mean()
kappa_score_cv = cohen_kappa_score(y_test, y_pred)
print("\n5-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", acc_score_cv)
print("Cohen's Kappa Score:", kappa_score_cv)

# 10-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=10)
acc_score_cv10 = cv_scores.mean()
kappa_score_cv10 = cohen_kappa_score(y_test, y_pred)
print("\n10-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", acc_score_cv10)
print("Cohen's Kappa Score:", kappa_score_cv10)

#10-fold stratified cross-validation (80/20 train/test split)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=skf)
acc_score_strat_cv = cv_scores.mean()
kappa_score_strat_cv = cohen_kappa_score(y_test, y_pred)
print("\n10-fold stratified cross-validation (80/20 train/test split):")
print("Accuracy Score:", acc_score_strat_cv)
print("Cohen's Kappa Score:", kappa_score_strat_cv)


Baseline Model:
Accuracy Score: 0.8981
Cohen's Kappa Score: 0.08653703500678733

Hold one out (90/10 train/test split):
Accuracy Score: 0.8956
Cohen's Kappa Score: 0.12325932544714624

5-fold cross-validation (80/20 train/test split):
Accuracy Score: 0.8885399999999999
Cohen's Kappa Score: 0.08653703500678733

10-fold cross-validation (80/20 train/test split):
Accuracy Score: 0.8911200000000001
Cohen's Kappa Score: 0.08653703500678733

10-fold stratified cross-validation (80/20 train/test split):
Accuracy Score: 0.89186
Cohen's Kappa Score: 0.08653703500678733


In [ ]:
params_dt = {'criterion': ['gini', 'entropy'], 'max_depth': [None, 10, 20, 30]}
grid_dt = GridSearchCV(DecisionTreeClassifier(), params_dt, cv=10, scoring='accuracy')
grid_dt.fit(X_train, y_train)
print("Best Decision Tree parameters:", grid_dt.best_params_)
print("Accuracy:", accuracy_score(y_test, grid_dt.predict(X_test)))

Best Decision Tree parameters: {'criterion': 'gini', 'max_depth': 10}
Accuracy: 0.9429


###**Naïve Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Define the Naive Bayes model
model = GaussianNB()

# ... (rest of the code is the same as the Decision Tree example, just replace the model definition)
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score

# Define features and target
X = clean_df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)',
            'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity', 'Bump',
            'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
            'Roundabout', 'Station', 'Stop',
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]
y = clean_df['Severity']

# Split the data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the decision tree model
model = GaussianNB()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy score and Cohen's kappa for the baseline model
NBacc_score = accuracy_score(y_test, y_pred)
NBkappa_score = cohen_kappa_score(y_test, y_pred)
print("Baseline Model:")
print("Accuracy Score:", NBacc_score)
print("Cohen's Kappa Score:", NBkappa_score)

# Hold one out (90/10 train/test split)
X_train_90, X_test_90, y_train_90, y_test_90 = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train_90, y_train_90)
y_pred_90 = model.predict(X_test_90)
NBacc_score_90 = accuracy_score(y_test_90, y_pred_90)
NBkappa_score_90 = cohen_kappa_score(y_test_90, y_pred_90)
print("\nHold one out (90/10 train/test split):")
print("Accuracy Score:", NBacc_score_90)
print("Cohen's Kappa Score:", NBkappa_score_90)

# 5-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=5)
NBacc_score_cv = cv_scores.mean()
NBkappa_score_cv = cohen_kappa_score(y_test, y_pred)
print("\n5-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", NBacc_score_cv)
print("Cohen's Kappa Score:", NBkappa_score_cv)

# 10-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=10)
NBacc_score_cv10 = cv_scores.mean()
NBkappa_score_cv10 = cohen_kappa_score(y_test, y_pred)
print("\n10-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", NBacc_score_cv10)
print("Cohen's Kappa Score:", NBkappa_score_cv10)

#10-fold stratified cross-validation (80/20 train/test split)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=skf)
NBacc_score_strat_cv = cv_scores.mean()
NBkappa_score_strat_cv = cohen_kappa_score(y_test, y_pred)
print("\n10-fold stratified cross-validation (80/20 train/test split):")
print("Accuracy Score:", NBacc_score_strat_cv)
print("Cohen's Kappa Score:", NBkappa_score_strat_cv)





Baseline Model:
Accuracy Score: 0.0307
Cohen's Kappa Score: 0.0006831603328556479

Hold one out (90/10 train/test split):
Accuracy Score: 0.0446
Cohen's Kappa Score: 0.00525401631033573

5-fold cross-validation (80/20 train/test split):
Accuracy Score: 0.043500000000000004
Cohen's Kappa Score: 0.0006831603328556479

10-fold cross-validation (80/20 train/test split):
Accuracy Score: 0.042899999999999994
Cohen's Kappa Score: 0.0006831603328556479

10-fold stratified cross-validation (80/20 train/test split):
Accuracy Score: 0.04182
Cohen's Kappa Score: 0.0006831603328556479


In [ ]:
params_nb = {'var_smoothing': [1e-10, 1e-9, 1e-8]}
grid_nb = GridSearchCV(GaussianNB(), params_nb, cv=10, scoring='accuracy')
grid_nb.fit(X_train, y_train)
print("Best Naïve Bayes parameters:", grid_nb.best_params_)
print("Accuracy:", accuracy_score(y_test, grid_nb.predict(X_test)))

Best Naïve Bayes parameters: {'var_smoothing': 1e-10}
Accuracy: 0.0308


###**Logistic regression**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score

# Define features and target
X = clean_df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)',
            'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity', 'Bump',
            'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
            'Roundabout', 'Station', 'Stop',
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]
y = clean_df['Severity']

# Split the data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the decision tree model with increased max_iter
model = LogisticRegression(solver = 'saga', random_state=42, max_iter=10000)

# # Define the decision tree model
# model = LogisticRegression(random_state=42)

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy score and Cohen's kappa for the baseline model
LRacc_score = accuracy_score(y_test, y_pred)
LRkappa_score = cohen_kappa_score(y_test, y_pred)
print("Baseline Model:")
print("Accuracy Score:", LRacc_score)
print("Cohen's Kappa Score:", LRkappa_score)

# Hold one out (90/10 train/test split)
X_train_90, X_test_90, y_train_90, y_test_90 = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train_90, y_train_90)
y_pred_90 = model.predict(X_test_90)
LRacc_score_90 = accuracy_score(y_test_90, y_pred_90)
LRkappa_score_90 = cohen_kappa_score(y_test_90, y_pred_90)
print("\nHold one out (90/10 train/test split):")
print("Accuracy Score:", LRacc_score_90)
print("Cohen's Kappa Score:", LRkappa_score_90)

# 5-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=5)
LRacc_score_cv = cv_scores.mean()
LRkappa_score_cv = cohen_kappa_score(y_test, y_pred)
print("\n5-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", LRacc_score_cv)
print("Cohen's Kappa Score:", LRkappa_score_cv)

# 10-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=10)
LRacc_score_cv10 = cv_scores.mean()
LRkappa_score_cv10 = cohen_kappa_score(y_test, y_pred)
print("\n10-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", LRacc_score_cv10)
print("Cohen's Kappa Score:", LRkappa_score_cv10)

#10-fold stratified cross-validation (80/20 train/test split)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=skf)
LRacc_score_strat_cv = cv_scores.mean()
LRkappa_score_strat_cv = cohen_kappa_score(y_test, y_pred)
print("\n10-fold stratified cross-validation (80/20 train/test split):")
print("Accuracy Score:", LRacc_score_strat_cv)
print("Cohen's Kappa Score:", LRkappa_score_strat_cv)



Baseline Model:
Accuracy Score: 0.9452
Cohen's Kappa Score: -0.00012319035647623267

Hold one out (90/10 train/test split):
Accuracy Score: 0.9452
Cohen's Kappa Score: 0.0

5-fold cross-validation (80/20 train/test split):
Accuracy Score: 0.9427999999999999
Cohen's Kappa Score: -0.00012319035647623267


In [ ]:
params_lr = {'penalty': ['l1', 'l2']}
grid_lr = GridSearchCV(LogisticRegression(max_iter=10000, solver='saga'), params_lr, cv=10, scoring='accuracy')
grid_lr.fit(X_train, y_train)
print("Best Logistic Regression parameters:", grid_lr.best_params_)
print("Accuracy:", accuracy_score(y_test, grid_lr.predict(X_test)))

###**Ramdon forest**

In [ ]:
# Define features and target
X = clean_df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)',
            'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity', 'Bump',
            'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
            'Roundabout', 'Station', 'Stop',
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]
y = clean_df['Severity']
# Split the data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the AdaBoost model
model = RandomForestClassifier(random_state=42)

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy score and Cohen's kappa for the baseline model
RFacc_score = accuracy_score(y_test, y_pred)
RFkappa_score = cohen_kappa_score(y_test, y_pred)
print("Baseline Model:")
print("Accuracy Score:", RFacc_score)
print("Cohen's Kappa Score:", RFkappa_score)

# Hold one out (90/10 train/test split)
X_train_90, X_test_90, y_train_90, y_test_90 = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train_90, y_train_90)
y_pred_90 = model.predict(X_test_90)
RFacc_score_90 = accuracy_score(y_test_90, y_pred_90)
RFkappa_score_90 = cohen_kappa_score(y_test_90, y_pred_90)
print("\nHold one out (90/10 train/test split):")
print("Accuracy Score:", RFacc_score_90)
print("Cohen's Kappa Score:", RFkappa_score_90)
# 5-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=5)
RFacc_score_cv = cv_scores.mean()
RFkappa_score_cv = cohen_kappa_score(y_test, y_pred)
print("\n5-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", RFacc_score_cv)
print("Cohen's Kappa Score:", RFkappa_score_cv)

# 10-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=10)
RFacc_score_cv10 = cv_scores.mean()
RFkappa_score_cv10 = cohen_kappa_score(y_test, y_pred)
print("\n10-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", RFacc_score_cv10)
print("Cohen's Kappa Score:", RFkappa_score_cv10)

#10-fold stratified cross-validation (80/20 train/test split)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=skf)
RFacc_score_strat_cv = cv_scores.mean()
RFkappa_score_strat_cv = cohen_kappa_score(y_test, y_pred)
print("\n10-fold stratified cross-validation (80/20 train/test split):")
print("Accuracy Score:", RFacc_score_strat_cv)
print("Cohen's Kappa Score:", RFkappa_score_strat_cv)




In [ ]:
params_rf = {'criterion': ['gini', 'entropy'], 'max_depth': [10, 20]}
grid_rf = GridSearchCV(RandomForestClassifier(), params_rf, cv=10, scoring='accuracy')
grid_rf.fit(X_train, y_train)
print("Best Random Forest parameters:", grid_rf.best_params_)
print("Accuracy:", accuracy_score(y_test, grid_rf.predict(X_test)))

###**Adaboost**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier


# Define features and target
X = clean_df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)',
            'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity', 'Bump',
            'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
            'Roundabout', 'Station', 'Stop',
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]
y = clean_df['Severity']
# Split the data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the AdaBoost model
model = AdaBoostClassifier(random_state=42)

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy score and Cohen's kappa for the baseline model
ABacc_score = accuracy_score(y_test, y_pred)
ABkappa_score = cohen_kappa_score(y_test, y_pred)
print("Baseline Model:")
print("Accuracy Score:", ABacc_score)
print("Cohen's Kappa Score:", ABkappa_score)

# Hold one out (90/10 train/test split)
X_train_90, X_test_90, y_train_90, y_test_90 = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train_90, y_train_90)
y_pred_90 = model.predict(X_test_90)
ABacc_score_90 = accuracy_score(y_test_90, y_pred_90)
ABkappa_score_90 = cohen_kappa_score(y_test_90, y_pred_90)
print("\nHold one out (90/10 train/test split):")
print("Accuracy Score:", ABacc_score_90)
print("Cohen's Kappa Score:", ABkappa_score_90)
# 5-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=5)
ABacc_score_cv = cv_scores.mean()
ABkappa_score_cv = cohen_kappa_score(y_test, y_pred)
print("\n5-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", ABacc_score_cv)
print("Cohen's Kappa Score:", ABkappa_score_cv)

# 10-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=10)
ABacc_score_cv10 = cv_scores.mean()
ABkappa_score_cv10 = cohen_kappa_score(y_test, y_pred)
print("\n10-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", ABacc_score_cv10)
print("Cohen's Kappa Score:", ABkappa_score_cv10)

#10-fold stratified cross-validation (80/20 train/test split)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=skf)
ABacc_score_strat_cv = cv_scores.mean()
ABkappa_score_strat_cv = cohen_kappa_score(y_test, y_pred)
print("\n10-fold stratified cross-validation (80/20 train/test split):")
print("Accuracy Score:", ABacc_score_strat_cv)
print("Cohen's Kappa Score:", ABkappa_score_strat_cv)



In [ ]:
params_ab = {'n_estimators': [10, 50, 100], 'learning_rate': [0.1, 0.5, 1]}
grid_ab = GridSearchCV(AdaBoostClassifier(), params_ab, cv=10, scoring='accuracy')
grid_ab.fit(X_train, y_train)
print("Best AdaBoost parameters:", grid_ab.best_params_)
print("Accuracy:", accuracy_score(y_test, grid_ab.predict(X_test)))

## Supervised Learning Models (predicting the possibility of an accident is severe)
Using machine learning models to predict the likelihood of a serious accident might enhance safety procedures and lessen the effects of accidents. Accidents can result in major issues including injuries, property damage, and even fatalities. Machine learning algorithms may be taught to precisely forecast the possibility of a catastrophic accident occurring by evaluating massive datasets of accident reports and detecting patterns and risk variables linked with severe accidents. The creation of safety measures and policies, such as bettering road design, traffic management, and emergency response, can be informed by this information in order to decrease the incidence and severity of accidents. Overall, the creation of machine learning models for anticipating the likelihood of serious mishaps can be very advantageous for the welfare and safety of the general population.



###**Decision tree**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Define features and target
X = clean_df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)',
            'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity', 'Bump',
            'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
            'Roundabout', 'Station', 'Stop',
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]
y = clean_df['severe_accident']

# Split the data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the decision tree model
model = DecisionTreeClassifier(random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy score and Cohen's kappa for the baseline model
acc_score = accuracy_score(y_test, y_pred)
kappa_score = cohen_kappa_score(y_test, y_pred)
print("Baseline Model:")
print("Accuracy Score:", acc_score)
print("Cohen's Kappa Score:", kappa_score)

# Hold one out (90/10 train/test split)
X_train_90, X_test_90, y_train_90, y_test_90 = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train_90, y_train_90)
y_pred_90 = model.predict(X_test_90)
Pacc_score_90 = accuracy_score(y_test_90, y_pred_90)
Pkappa_score_90 = cohen_kappa_score(y_test_90, y_pred_90)
print("\nHold one out (90/10 train/test split):")
print("Accuracy Score:", Pacc_score_90)
print("Cohen's Kappa Score:", Pkappa_score_90)

# 5-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=5)
Pacc_score_cv = cv_scores.mean()
Pkappa_score_cv = cohen_kappa_score(y_test, y_pred)
print("\n5-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", Pacc_score_cv)
print("Cohen's Kappa Score:", Pkappa_score_cv)

# 10-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=10)
Pacc_score_cv10 = cv_scores.mean()
Pkappa_score_cv10 = cohen_kappa_score(y_test, y_pred)
print("\n10-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", Pacc_score_cv10)
print("Cohen's Kappa Score:", Pkappa_score_cv10)

#10-fold stratified cross-validation (80/20 train/test split)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=skf)
Pacc_score_strat_cv = cv_scores.mean()
Pkappa_score_strat_cv = cohen_kappa_score(y_test, y_pred)
print("\n10-fold stratified cross-validation (80/20 train/test split):")
print("Accuracy Score:", Pacc_score_strat_cv)
print("Cohen's Kappa Score:", Pkappa_score_strat_cv)


In [ ]:
params_dt = {'criterion': ['gini', 'entropy'], 'max_depth': [None, 10, 20, 30]}
grid_dt = GridSearchCV(DecisionTreeClassifier(), params_dt, cv=10, scoring='accuracy')
grid_dt.fit(X_train, y_train)
print("Best Decision Tree parameters:", grid_dt.best_params_)
print("Accuracy:", accuracy_score(y_test, grid_dt.predict(X_test)))

###**Naïve Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Define the Naive Bayes model
model = GaussianNB()

# ... (rest of the code is the same as the Decision Tree example, just replace the model definition)
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score

# Define features and target
X = clean_df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)',
            'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity', 'Bump',
            'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
            'Roundabout', 'Station', 'Stop',
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]
y = clean_df['severe_accident']

# Split the data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the decision tree model
model = GaussianNB()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy score and Cohen's kappa for the baseline model
PNBacc_score = accuracy_score(y_test, y_pred)
PNBkappa_score = cohen_kappa_score(y_test, y_pred)
print("Baseline Model:")
print("Accuracy Score:", PNBacc_score)
print("Cohen's Kappa Score:", PNBkappa_score)

# Hold one out (90/10 train/test split)
X_train_90, X_test_90, y_train_90, y_test_90 = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train_90, y_train_90)
y_pred_90 = model.predict(X_test_90)
PNBacc_score_90 = accuracy_score(y_test_90, y_pred_90)
PNBkappa_score_90 = cohen_kappa_score(y_test_90, y_pred_90)
print("\nHold one out (90/10 train/test split):")
print("Accuracy Score:", PNBacc_score_90)
print("Cohen's Kappa Score:", PNBkappa_score_90)

# 5-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=5)
PNBacc_score_cv = cv_scores.mean()
PNBkappa_score_cv = cohen_kappa_score(y_test, y_pred)
print("\n5-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", PNBacc_score_cv)
print("Cohen's Kappa Score:", PNBkappa_score_cv)

# 10-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=10)
PNBacc_score_cv10 = cv_scores.mean()
PNBkappa_score_cv10 = cohen_kappa_score(y_test, y_pred)
print("\n10-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", PNBacc_score_cv10)
print("Cohen's Kappa Score:", PNBkappa_score_cv10)

#10-fold stratified cross-validation (80/20 train/test split)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=skf)
PNBacc_score_strat_cv = cv_scores.mean()
PNBkappa_score_strat_cv = cohen_kappa_score(y_test, y_pred)
print("\n10-fold stratified cross-validation (80/20 train/test split):")
print("Accuracy Score:", PNBacc_score_strat_cv)
print("Cohen's Kappa Score:", PNBkappa_score_strat_cv)





In [ ]:
params_nb = {'var_smoothing': [1e-10, 1e-9, 1e-8]}
grid_nb = GridSearchCV(GaussianNB(), params_nb, cv=10, scoring='accuracy')
grid_nb.fit(X_train, y_train)
print("Best Naïve Bayes parameters:", grid_nb.best_params_)
print("Accuracy:", accuracy_score(y_test, grid_nb.predict(X_test)))

###**Logistic regression**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score

# Define features and target
X = clean_df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)',
            'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity', 'Bump',
            'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
            'Roundabout', 'Station', 'Stop',
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]
y = clean_df['severe_accident']

# Split the data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the decision tree model with increased max_iter
model = LogisticRegression(random_state=42, max_iter=1000)

# # Define the decision tree model
# model = LogisticRegression(random_state=42)

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy score and Cohen's kappa for the baseline model
PLRacc_score = accuracy_score(y_test, y_pred)
PLRkappa_score = cohen_kappa_score(y_test, y_pred)
print("Baseline Model:")
print("Accuracy Score:", PLRacc_score)
print("Cohen's Kappa Score:", PLRkappa_score)

# Hold one out (90/10 train/test split)
X_train_90, X_test_90, y_train_90, y_test_90 = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train_90, y_train_90)
y_pred_90 = model.predict(X_test_90)
PLRacc_score_90 = accuracy_score(y_test_90, y_pred_90)
PLRkappa_score_90 = cohen_kappa_score(y_test_90, y_pred_90)
print("\nHold one out (90/10 train/test split):")
print("Accuracy Score:", PLRacc_score_90)
print("Cohen's Kappa Score:", PLRkappa_score_90)

# 5-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=5)
PLRacc_score_cv = cv_scores.mean()
PLRkappa_score_cv = cohen_kappa_score(y_test, y_pred)
print("\n5-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", PLRacc_score_cv)
print("Cohen's Kappa Score:", PLRkappa_score_cv)

# 10-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=10)
PLRacc_score_cv10 = cv_scores.mean()
PLRkappa_score_cv10 = cohen_kappa_score(y_test, y_pred)
print("\n10-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", PLRacc_score_cv10)
print("Cohen's Kappa Score:", PLRkappa_score_cv10)

#10-fold stratified cross-validation (80/20 train/test split)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=skf)
PLRacc_score_strat_cv = cv_scores.mean()
PLRkappa_score_strat_cv = cohen_kappa_score(y_test, y_pred)
print("\n10-fold stratified cross-validation (80/20 train/test split):")
print("Accuracy Score:", PLRacc_score_strat_cv)
print("Cohen's Kappa Score:", PLRkappa_score_strat_cv)



In [ ]:
params_lr = {'penalty': ['l1', 'l2']}
grid_lr = GridSearchCV(LogisticRegression(max_iter=1000), params_lr, cv=10, scoring='accuracy')
grid_lr.fit(X_train, y_train)
print("Best Logistic Regression parameters:", grid_lr.best_params_)
print("Accuracy:", accuracy_score(y_test, grid_lr.predict(X_test)))

###**Ramdon forest**

In [ ]:
# Define features and target
X = clean_df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)',
            'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity', 'Bump',
            'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
            'Roundabout', 'Station', 'Stop',
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]
y = clean_df['severe_accident']
# Split the data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the AdaBoost model
model = RandomForestClassifier(random_state=42)

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy score and Cohen's kappa for the baseline model
PRFacc_score = accuracy_score(y_test, y_pred)
PRFkappa_score = cohen_kappa_score(y_test, y_pred)
print("Baseline Model:")
print("Accuracy Score:", PRFacc_score)
print("Cohen's Kappa Score:", PRFkappa_score)

# Hold one out (90/10 train/test split)
X_train_90, X_test_90, y_train_90, y_test_90 = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train_90, y_train_90)
y_pred_90 = model.predict(X_test_90)
PRFacc_score_90 = accuracy_score(y_test_90, y_pred_90)
PRFkappa_score_90 = cohen_kappa_score(y_test_90, y_pred_90)
print("\nHold one out (90/10 train/test split):")
print("Accuracy Score:", acc_score_90)
print("Cohen's Kappa Score:", kappa_score_90)
# 5-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=5)
PRFacc_score_cv = cv_scores.mean()
PRFkappa_score_cv = cohen_kappa_score(y_test, y_pred)
print("\n5-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", PRFacc_score_cv)
print("Cohen's Kappa Score:", PRFkappa_score_cv)

# 10-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=10)
PRFacc_score_cv10 = cv_scores.mean()
PRFkappa_score_cv10 = cohen_kappa_score(y_test, y_pred)
print("\n10-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", PRFacc_score_cv10)
print("Cohen's Kappa Score:", PRFkappa_score_cv10)

#10-fold stratified cross-validation (80/20 train/test split)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=skf)
PRFacc_score_strat_cv = cv_scores.mean()
PRFkappa_score_strat_cv = cohen_kappa_score(y_test, y_pred)
print("\n10-fold stratified cross-validation (80/20 train/test split):")
print("Accuracy Score:", PRFacc_score_strat_cv)
print("Cohen's Kappa Score:", PRFkappa_score_strat_cv)




In [ ]:
params_rf = {'criterion': ['gini', 'entropy'], 'max_depth': [10, 20]}
grid_rf = GridSearchCV(RandomForestClassifier(), params_rf, cv=10, scoring='accuracy')
grid_rf.fit(X_train, y_train)
print("Best Random Forest parameters:", grid_rf.best_params_)
print("Accuracy:", accuracy_score(y_test, grid_rf.predict(X_test)))

###**Adaboost**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier


# Define features and target
X = clean_df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)',
            'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity', 'Bump',
            'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
            'Roundabout', 'Station', 'Stop',
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]
y = clean_df['severe_accident']
# Split the data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the AdaBoost model
model = AdaBoostClassifier(random_state=42)

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy score and Cohen's kappa for the baseline model
PABacc_score = accuracy_score(y_test, y_pred)
PABkappa_score = cohen_kappa_score(y_test, y_pred)
print("Baseline Model:")
print("Accuracy Score:", PABacc_score)
print("Cohen's Kappa Score:", PABkappa_score)

# Hold one out (90/10 train/test split)
X_train_90, X_test_90, y_train_90, y_test_90 = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train_90, y_train_90)
y_pred_90 = model.predict(X_test_90)
PABacc_score_90 = accuracy_score(y_test_90, y_pred_90)
PABkappa_score_90 = cohen_kappa_score(y_test_90, y_pred_90)
print("\nHold one out (90/10 train/test split):")
print("Accuracy Score:", PABacc_score_90)
print("Cohen's Kappa Score:", PABkappa_score_90)
# 5-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=5)
PABacc_score_cv = cv_scores.mean()
PABkappa_score_cv = cohen_kappa_score(y_test, y_pred)
print("\n5-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", PABacc_score_cv)
print("Cohen's Kappa Score:", PABkappa_score_cv)

# 10-fold cross-validation (80/20 train/test split)
cv_scores = cross_val_score(model, X, y, cv=10)
PABacc_score_cv10 = cv_scores.mean()
PABkappa_score_cv10 = cohen_kappa_score(y_test, y_pred)
print("\n10-fold cross-validation (80/20 train/test split):")
print("Accuracy Score:", PABacc_score_cv10)
print("Cohen's Kappa Score:", PABkappa_score_cv10)

#10-fold stratified cross-validation (80/20 train/test split)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=skf)
PABacc_score_strat_cv = cv_scores.mean()
PABkappa_score_strat_cv = cohen_kappa_score(y_test, y_pred)
print("\n10-fold stratified cross-validation (80/20 train/test split):")
print("Accuracy Score:", PABacc_score_strat_cv)
print("Cohen's Kappa Score:", PABkappa_score_strat_cv)



In [ ]:
params_ab = {'n_estimators': [10, 50, 100], 'learning_rate': [0.1, 0.5, 1]}
grid_ab = GridSearchCV(AdaBoostClassifier(), params_ab, cv=10, scoring='accuracy')
grid_ab.fit(X_train, y_train)
print("Best AdaBoost parameters:", grid_ab.best_params_)
print("Accuracy:", accuracy_score(y_test, grid_ab.predict(X_test)))

# Part 4: Model Comparsion and Conclusion

In [ ]:
# To compare the performance of the different models, you can use the same evaluation metrics (accuracy and Cohen's Kappa score) that you have used for the Decision Tree model. After implementing each of the other models (Logistic Regression, Naïve Bayes, Random Forest, and Adaboost) and applying cross-validation, you can summarize the results in a table or plot.

# Here's a suggested approach to compare the models:

# Create a dictionary or DataFrame to store the results for each model.
# For each model, implement the code similar to what you provided for the Decision Tree model.
# Store the accuracy and Cohen's Kappa scores for each model in the results dictionary or DataFrame.
# After calculating the scores for all models, compare them using the mean values for each evaluation metric.
# Choose the model with the highest mean accuracy and/or Cohen's Kappa score as the best-performing model.
# Optionally, you can visualize the results in a bar plot or another suitable plot to make the comparison more intuitive.
# Here's an example of how to create a dictionary to store the results:

results = {
    "Model": [],
    "Accuracy": [],
    "Cohen's Kappa": []
}
# After computing the accuracy and Cohen's Kappa scores for each model, you can append the results to the dictionary like this:

results["Model"].append("Decision Tree")
results["Accuracy"].append(acc_score_cv10)  # Use the desired accuracy score (e.g., 10-fold CV)
results["Cohen's Kappa"].append(kappa_score_cv10)  # Use the desired Cohen's Kappa score (e.g., 10-fold CV)
results["Model"].append("Naive Bayes")
results["Accuracy"].append(NBacc_score_cv10)  # Use the desired accuracy score (e.g., 10-fold CV)
results["Cohen's Kappa"].append(NBkappa_score_cv10)
results["Model"].append("Logistic Regression")
results["Accuracy"].append(LRacc_score_cv10)  # Use the desired accuracy score (e.g., 10-fold CV)
results["Cohen's Kappa"].append(LRkappa_score_cv10)
results["Model"].append("Random Forest")
results["Accuracy"].append(RFacc_score_cv10)  # Use the desired accuracy score (e.g., 10-fold CV)
results["Cohen's Kappa"].append(RFkappa_score_cv10)
results["Model"].append("Adaboost")
results["Accuracy"].append(ABacc_score_cv)  # Use the desired accuracy score (e.g., 10-fold CV)
results["Cohen's Kappa"].append(ABkappa_score_cv10)
# Repeat the same for the other models (Logistic Regression, Naïve Bayes, Random Forest, and Adaboost).

# After obtaining the results for all models, convert the dictionary into a DataFrame:

results_df = pd.DataFrame(results)

# Then, you can sort the DataFrame based on the evaluation metric of your choice (accuracy or Cohen's Kappa) to find the best-performing model:
results_df.sort_values(by="Accuracy", ascending=False, inplace=True)

# Finally, visualize the results using a bar plot or any other suitable plot:
#import matplotlib.pyplot as plt

plt.bar(results_df["Model"], results_df["Accuracy"])
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.title("Model Comparison - Accuracy on predicting accident severity")
plt.xticks(rotation=90)
for i, v in enumerate(results_df["Accuracy"]):
    plt.text(i, v, str(round(v, 3)), color='blue', fontweight='bold')
plt.show()



In [ ]:
possibility_results = {
    "Model": [],
    "Accuracy": [],
    "Cohen's Kappa": []
}
# After computing the accuracy and Cohen's Kappa scores for each model, you can append the results to the dictionary like this:

possibility_results["Model"].append("Decision Tree")
possibility_results["Accuracy"].append(Pacc_score_cv10)  # Use the desired accuracy score (e.g., 10-fold CV)
possibility_results["Cohen's Kappa"].append(Pkappa_score_cv10)  # Use the desired Cohen's Kappa score (e.g., 10-fold CV)
possibility_results["Model"].append("Naive Bayes")
possibility_results["Accuracy"].append(PNBacc_score_cv10)  # Use the desired accuracy score (e.g., 10-fold CV)
possibility_results["Cohen's Kappa"].append(PNBkappa_score_cv10)
possibility_results["Model"].append("Logistic Regression")
possibility_results["Accuracy"].append(PLRacc_score_cv10)  # Use the desired accuracy score (e.g., 10-fold CV)
possibility_results["Cohen's Kappa"].append(PLRkappa_score_cv10)
possibility_results["Model"].append("Random Forest")
possibility_results["Accuracy"].append(PRFacc_score_cv10)  # Use the desired accuracy score (e.g., 10-fold CV)
possibility_results["Cohen's Kappa"].append(PRFkappa_score_cv10)
possibility_results["Model"].append("Adaboost")
possibility_results["Accuracy"].append(PABacc_score_cv)  # Use the desired accuracy score (e.g., 10-fold CV)
possibility_results["Cohen's Kappa"].append(PABkappa_score_cv10)
# Repeat the same for the other models (Logistic Regression, Naïve Bayes, Random Forest, and Adaboost).

# After obtaining the results for all models, convert the dictionary into a DataFrame:

possibility_results_df = pd.DataFrame(possibility_results)

# Then, you can sort the DataFrame based on the evaluation metric of your choice (accuracy or Cohen's Kappa) to find the best-performing model:
possibility_results_df.sort_values(by="Accuracy", ascending=False, inplace=True)

# Finally, visualize the results using a bar plot or any other suitable plot:
#import matplotlib.pyplot as plt

plt.bar(possibility_results_df["Model"], possibility_results_df["Accuracy"])
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.title("Model Comparison - Accuracy on predicting the possibility of severe accident")
plt.xticks(rotation=90)
for i, v in enumerate(possibility_results_df["Accuracy"]):
    plt.text(i, v, str(round(v, 3)), color='blue', fontweight='bold')
plt.show()


In [ ]:
possibility_results_df#display the accuracy and kappa score of all 5 models

In this project, we want to implement both unsupervised and supervised machine learning models to achieve our research questions. We first cleaned the dataset and ensure the data without any outliers (`accident_time` less than 0) and Null values. We then created a new column called `severe_accident` that can distinguish whether an accident is severe or not. According to our dataset, our two labels are binary variables so the machine learning will be a classification question.

We used PCA (unsupervised learning) to reduce dimensional and apply supervised learning models (Logistic Regression with regularizations) after that. We then used supervised laerning methods like decision tree, random forest, SVM, Adaboost and Naive Bayes. We also tunned their hyperparameters to see which one can produce the best accuracy.

After using PCA on the feature list, the best model is logistic regression with l2 and elastic net.
for the model that predicting the severity of an accident, the best one is random forest, the accuracy is 94.4%.
for the model that predicting the possibility of an accident is severe or not, the best one is adaboost with accuracy of 95.446%

# Part 5 : Challenges and Obstacles Faced

* **Missing Data:** There are many missing value/nulls in data, due to collection error or something else. In this way, the cleaning process has eliminated many rows.

* **Data size:** This dataset is extremely large and has nearly 3 million rows. There are still nearly 1 million rows after the data preprocessing and cleaning phase. In order to avoid system overload we have to using sampling to select 50000 rows from the dataset but we think those data might be not very representative for the whole picture.

* **Select features:** Additionally, we would like to add some features that can make our model better and representative but the colab environment is unable to process too many features in short time. So we have to further drop the number of features and only keep 18 of them.

# Part 6 : Further Exploration

* We would like to try methods covered in the lecture to replace missing value/Nulls with aggregation or KNN value to retain more rows in training data
* We would like to access more powerful system that can process more rows where we can expand the size of our training and testing data. The current environment has very limited RAM which always crash for large dataset.